# Web Scraping Part 2 (Jobstreet)
## Done by: Yann
### Source: https://www.jobstreet.com.sg/data-analyst-jobs/in-Singapore?pg=1

#### The difference in Part 2 is that I'm trying to scrape information from multiple pages

##

### Import Libraries

In [1]:
import csv
import requests
from bs4 import BeautifulSoup

##

### Specify base URL

In [3]:
base_url = 'https://www.jobstreet.com.sg/data-analyst-jobs/in-Singapore?pg={}'

##

In [5]:
# Specify the total number of pages you want to scrape
total_pages = 5

data_rows = []

for page in range(1, total_pages + 1):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    job_listings = soup.find_all('div', class_='z1s6m00 _1hbhsw67i _1hbhsw66e _1hbhsw69q _1hbhsw68m _1hbhsw6n _1hbhsw65a _1hbhsw6ga _1hbhsw6fy')

    for job in job_listings:    
        # Get Title
        title = job.find('span', class_='z1s6m00').text.strip()

        # Get company name
        company = job.find('a', class_='_6xa4xb0 z1s6m00 z1s6m0f rqoqz4').text.strip()  

        # Get Salary info
        salary_element = job.find('span', class_='z1s6m00 _1hbhsw64y y44q7i0 y44q7i3 y44q7i21 y44q7ih')    
        if salary_element and salary_element.find_next_sibling('span', class_='z1s6m00 _1hbhsw64y y44q7i0 y44q7i3 y44q7i21 y44q7ih'):
            salary = salary_element.find_next_sibling('span', class_='z1s6m00 _1hbhsw64y y44q7i0 y44q7i3 y44q7i21 y44q7ih').text.strip()
        else:
            salary = 'No Salary Displayed'
        
        # Get description    
        selling_points = job.find('div', class_='z1s6m00 _1hbhsw6ba _1hbhsw64y')
        if selling_points:
            points = selling_points.find_all('span', class_='z1s6m00 _1hbhsw64y y44q7i0 y44q7i1 y44q7i21 _1d0g9qk4 y44q7i7')
            if points:
                selling_points_text = ', '.join(point.get_text() for point in points)
            else:
                selling_points_text = 'No selling points available'
        else:
            selling_points_text = 'No selling points available'


        # Get date posted
        exact_date = job.find('time', class_='z1s6m00 _1hbhsw64y')
        date_posted = exact_date['datetime'].split('T')[0] if exact_date else 'No Date Available'
        how_long_ago = job.find('span', class_='z1s6m00 _1hbhsw64y y44q7i0 y44q7i1 y44q7i22 y44q7ih').text.strip()

        data_rows.append([title, company, salary, selling_points_text, date_posted, how_long_ago])

# Write the data to CSV file
with open('job_listings(Multiple).csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Title', 'Company', 'Salary', 'Description', 'Date Posted', 'Days Elapsed'])
    writer.writerows(data_rows)

### Things to note
##### 1 page will return me 30 results. So n pages will result in 30n pages
##### I'm using this information to check if I'm getting the right number of results